In [5]:
import spacy
import os
from google.colab import drive
drive.mount('/content/drive')
os.listdir('/content/drive/MyDrive/Colab Notebooks')
nlp = spacy.load('en_core_web_sm')

ModuleNotFoundError: No module named 'google.colab'

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model



pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

ValueError: Value must be a nonnegative integer or None

In [9]:
lines=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/english-japanese.csv",encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/english-japanese.csv'

In [ ]:
lines=lines[['japanese','english']]

In [ ]:
lines.head(20)

,japanese,english
0,誰が一番に着くか私には分かりません。,i can 't tell who will arrive first .
1,多くの動物が人間によって滅ぼされた。,many animals have been destroyed by men .
2,私はテニス部員です。,i 'm in the tennis club .
3,エミは幸せそうに見えます。,emi looks happy .
4,この事実を心に留めておいて下さい。,please bear this fact in mind .
5,彼女は私たちの世話をしてくれる。,she takes care of my children .
6,私達は国際人になりたいと思います。,we want to be international .
7,約束を破るべきではありません。,you ought not to break your promise .
8,道路を横切るときは車に注意しなさい。,"when you cross the street , watch out for cars ."
9,私には生き甲斐がない。,i have nothing to live for .


In [ ]:
pd.isnull(lines).sum()

japanese    0
english     0
dtype: int64

In [ ]:
lines=lines[~pd.isnull(lines['english'])]

In [ ]:
lines.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [ ]:
lines=lines.sample(n=250,random_state=42)
lines.shape

(250, 2)

In [ ]:
# Lowercase all characters
lines['english']=lines['english'].apply(lambda x: x.lower())
lines['japanese']=lines['japanese'].apply(lambda x: x.lower())

In [ ]:
# Remove quotes
lines['english']=lines['english'].apply(lambda x: re.sub("'", '', x))
lines['japanese']=lines['japanese'].apply(lambda x: re.sub("'", '', x))

In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['japanese']=lines['japanese'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english']=lines['english'].apply(lambda x: x.translate(remove_digits))
lines['japanese']=lines['japanese'].apply(lambda x: x.translate(remove_digits))

lines['japanese'] = lines['japanese'].apply(lambda x: re.sub("[1234567890]", "", x))

# Remove extra spaces
lines['english']=lines['english'].apply(lambda x: x.strip())
lines['japanese']=lines['japanese'].apply(lambda x: x.strip())
lines['english']=lines['english'].apply(lambda x: re.sub(" +", " ", x))
lines['japanese']=lines['japanese'].apply(lambda x: re.sub(" +", " ", x))


In [ ]:
# Add start and end tokens to target sequences
lines['japanese'] = lines['japanese'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines.head()

,japanese,english
16397,START_ おじいさんはとてもゆっくりと話す。 _END,grandfather speaks very slowly
27071,START_ 私は彼の言ったことに反対する。 _END,i dissent from what he said
23443,START_ 食べる前に手を洗いなさい。 _END,wash your hands before eating
19970,START_ 彼は泳ぐのが好きです。 _END,he is fond of swimming
44751,START_ 彼は塀にもたれて立っていた。 _END,he stood against the wall


In [ ]:

all_eng_words=set()
for eng in lines['english']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_japanese_words=set()
for jap in lines['japanese']:
    cur=jap.split()
    cur=[cur[0]]+list(cur[1])+[cur[-1]]
    for word in cur:
        if word not in all_japanese_words:
            all_japanese_words.add(word)

In [1]:
len(all_eng_words)

NameError: name 'all_eng_words' is not defined

In [ ]:
len(all_japanese_words)

504

In [ ]:
lines['length_eng_sentence']=lines['english'].apply(lambda x:len(x.split(" ")))
lines['length_jap_sentence']=lines['japanese'].apply(lambda x:len(x.split(" ")[1])+2)

In [ ]:
lines.head()

,japanese,english,length_eng_sentence,length_jap_sentence
16397,START_ おじいさんはとてもゆっくりと話す。 _END,grandfather speaks very slowly,4,19
27071,START_ 私は彼の言ったことに反対する。 _END,i dissent from what he said,6,17
23443,START_ 食べる前に手を洗いなさい。 _END,wash your hands before eating,5,15
19970,START_ 彼は泳ぐのが好きです。 _END,he is fond of swimming,5,13
44751,START_ 彼は塀にもたれて立っていた。 _END,he stood against the wall,5,16


In [ ]:
lines[lines['length_eng_sentence']>30].shape

(0, 4)

In [ ]:
lines=lines[lines['length_eng_sentence']<=20]

In [ ]:
lines.shape

(250, 4)

In [ ]:
print("maximum length of Japanese Sentence ",max(lines['length_jap_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Japanese Sentence  26
maximum length of English Sentence  11


In [ ]:
max_length_src=max(lines['length_eng_sentence'])
max_length_tar=max(lines['length_jap_sentence'])

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_japanese_words))
num_encoder_tokens = len(all_eng_words)+100
num_decoder_tokens = len(all_japanese_words)+100
num_encoder_tokens, num_decoder_tokens

(657, 604)

In [ ]:
num_decoder_tokens += 1 #for zero padding


In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines = shuffle(lines)
lines.head(10)

,japanese,english,length_eng_sentence,length_jap_sentence
47579,START_ そのピアニストは皆に知られている。 _END,the name of the pianist is known to everybody,9,19
9935,START_ ひどい頭痛がします。 _END,i have a bad headache,5,12
19653,START_ 人の価値はその人の外観とは無関係だ。 _END,a man s worth has nothing to do with his appearance,11,20
32230,START_ その小説はロビンソン氏が書いたものではなかった。 _END,mr robinson didn t write the novel,7,26
1566,START_ 私はすぐに新しい学校に溶け込んだ。 _END,i quickly adapted myself to my new school,8,19
8241,START_ 彼は１日中黙っていた。 _END,he kept silent all day long,6,13
18086,START_ よく映画を見に行きます。 _END,i often go to the movies,6,14
31961,START_ 自転車を借りてもいいですか。 _END,may i borrow your bicycle,5,16
29023,START_ あなたは医者に見てもらった方がいい。 _END,you d better see a doctor,6,20
12140,START_ 彼は長い病気の後で、やつれて見えた。 _END,he cut a poor figure after his long illness,9,20


### Split the data into train and test

In [ ]:
X, y = lines['english'], lines['japanese']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((200,), (50,))

### Let us save this data

In [ ]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                target=target_text.split()
                target_text=[target[0]]+list(target[1])+[target[-1]]
                # decoder_target_data is ahead of decoder_input
                for t, word in enumerate(target_text):
                    if t<len(target_text)-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [ ]:
latent_dim=300

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 input_16 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 embedding_10 (Embedding)    (None, None, 300)            197100    ['input_15[0][0]']            
                                                                                                  
 embedding_11 (Embedding)    (None, None, 300)            181500    ['input_16[0][0]']            
                                                                                            

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 1
epochs = 1

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



<ipython-input-340-a780c545a7a7>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


200/200 [==============================] - 36s 117ms/step - loss: 4.7683 - val_loss: 4.2943


In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']



    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        sampled_char = reverse_target_char_index[sampled_token_index]
        #print(sampled_char)
        decoded_sentence +=sampled_char


        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)

print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Japanese Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Japanese Translation:', decoded_sentence[:-4])


1/1 [==============================] - 0s 25ms/step
Input English sentence: she cut me dead in the street
Actual Japanese Translation:  彼女は道で会っても僕をわざと無視した。 
Predicted Japanese Translation: 彼はははははににににい。
